In [ ]:
import h5py
import os
import numpy as np
import json

DATA_DIR = "../data/raw/"
TRAIN_DIR = os.path.join(DATA_DIR,"train")

## Caution
아래의 스크립트를 실행하기 위해서는 다음과 같은 폴더 구성을 지켜주셔야 합니다. 
```
|- data/
     |- raw/
         |- train/
              |- train.chunk.01~09
         |- test/
              |- test.chunk.01~02
         |- dev/
              |- dev.chunk.01
         |- cate1.json
```

이 코드들은 간단히 데이터를 까보기 위함일 뿐, 그렇게 의미있진 않을 겁니다. 
바로 데이터를 분석해보기 위해서는 저희가 직접 전처리한 데이터들을 다운 받아서 쓰시면 될 겁니다.

`3) 데이터 로더 만들기.ipynb`부터 참고해주세요

## Objective

    우리의 카카오가 어떤 데이터를 우리에게 넘겨주셨는지 한번 보아요.
    주어진 데이터가 어떤 꼴인지를 확인 좀 해봅시다

In [ ]:
f = h5py.File(os.path.join(TRAIN_DIR,"train.chunk.01"),'r')
print(list(f['train'].keys()))

## 주어진 데이터셋

| 컬럼  | 타입  | 설명  | 예시|
|---|---|---| ---|
| pid  | string | 상품 ID | O4486751463 |
| product  | string  | 상품명  | 직소퍼즐 - 1000조각 바다거북의 여행 (PL1275) |
| brand  | string  | 브랜드명  | 퍼즐라이프 |
| model  | string  | 정제된 상품명  | 퍼즐라이프 직소퍼즐 바다거북의 여행 |
| maker  | string  | 제조사   | ㈜크리스패션 |
| price  | int32  | 가격 | 16520 |
| updttm |  string | 상품정보업데이트 시간 | 20180227091029 |
| bcateid | int32  | 대카테고리 ID  | 1 |
| mcateid | int32  | 중카테고리 ID  | 1 |
| scateid | int32  | 소카테고리 ID  | 2 |
| dcateid | int32  | 세카테고리 ID  | -1 |
| img_feat | float32 | ResNet50(imagenet)의 모델의 출력  |array([...,..]) |

In [ ]:
def list_up_string_in_file(column_name, nums=20):
    # h5py의 column_name을 열어보는 코드
    return list(map(lambda x: x.decode('utf-8'), f['train'][column_name][:nums]))

### pid

상품 ID에는 정해진 길이의 string으로 된 구조가 있는 ID이다.

가설 1) 
> 브랜드명, 상품명, 제조사, 카테고리 등의 정보로 조합되었을 것이다.

In [ ]:
list_up_string_in_file(column_name="pid", nums=30)

### product & brand & model & maker

상품 이름과 브랜드와 model의 관계를 분석해 볼 필요가 있다. 


가설 1) 
> 브랜드와 maker 사이의 관계 

가설 2) 
> 상품 이름과 model 사이의 관계

In [ ]:
products = list_up_string_in_file('product')
brands = list_up_string_in_file('brand')
models = list_up_string_in_file('model')
makers = list_up_string_in_file('maker')
for p_name, b_name, m_name, ma_name in zip(products, brands, models, makers):
    print("product : ", p_name)
    print("brand : ", b_name)
    print("model : ", m_name)
    print("maker : ", ma_name)
    print("---------------------")

### Price

가격은 정수로 존재한다


가설 1) 
> 존재하지 않은 경우, -1로 되어 있는 것으로 추정된다

In [ ]:
list(f['train']['price'][:20])

### updttm	

입력 시간은 중요하다고 느껴지지 않지만, 확인해볼 가치는 있다.

가설
> 20180227091029 : 2017년 02월 27일 09시 10분 29초 의 구조로 되어 있을 것이다

In [ ]:
updttms = list_up_string_in_file('updttm')
for updttm in updttms :
    print(updttm)

### Category ID

카테고리 아이디도 정수 값으로 존재한다

가설 1) 
> 존재하지 않는 경우 -1로 되어 있는 것으로 추정된다.


In [ ]:
bcateids = list(f['train']['bcateid'][:20])
mcateids = list(f['train']['mcateid'][:20])
scateids = list(f['train']['scateid'][:20])
dcateids = list(f['train']['dcateid'][:20])
print("[  대, 중,  소, 세]")
for b_id, m_id, s_id, d_id in zip(bcateids, mcateids, scateids, dcateids):
    print("[{:3d},{:3d},{:3d},{:3d}]".format(b_id, m_id,s_id, d_id))
    

카테고리 정보는 따로 json 파일로 제공되었다.

In [ ]:
json_data = open(os.path.join(DATA_DIR,'cate1.json')).read()
data = json.loads(json_data)

데이터는 key: 파일이름 - value : id로 구성되어 있는데, 

역으로 우리가 바꾸기 위해서는

key와 value를 서로 반대로 바꾸어주는 게 편하다

In [ ]:
# b: 대분류 m: 중분류 s: 소분류 d: 세분류
b_cate_map = {value : key for key, value in data['b'].items()}
m_cate_map = {value : key for key, value in data['m'].items()}
s_cate_map = {value : key for key, value in data['s'].items()}
d_cate_map = {value : key for key, value in data['d'].items()}


In [ ]:
print("[  대, 중,  소, 세]")
for b_id, m_id, s_id, d_id in zip(bcateids, mcateids, scateids, dcateids):
    print("[{},{},{},{}]".format(b_cate_map[b_id],
                                 m_cate_map[m_id],
                                 s_cate_map[s_id], 
                                 d_cate_map[d_id]))
    